In [4]:
# !pip install deep_translator
# from deep_translator import GoogleTranslator
import random
import numpy as np
import regex
import re
from os import listdir
from os.path import isfile, join
import requests
# import pyttsx3
# from profanity_filter import ProfanityFilter

# !pip install python_telegram_bot

import logging
import soundfile as sf

from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, Update
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler,
    CallbackContext,
    CallbackQueryHandler
)
from telegram import InlineKeyboardButton, InlineKeyboardMarkup

import sys
import os
import pandas as pd
import subprocess
import json
import codecs
import unidecode

In [5]:
with open('/media/boris/F/token.txt') as token_file:
    bot_token = token_file.readline().replace('\n', '')

In [ ]:
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)

TRANSLATE_EN_RU, TRANSLATE_RU_EN, SYNTHESIZE_EN = range(3)

def normalize_str(txt) -> str:
    # TODO: REPLACE WITH YOUR OWN NORMALIZATION LOGIC HERE!!!!   
    valid_chars = (" ", "'", "!", ".", "?", "&",
                   "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
                   "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z",
                   "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M",
                   "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z",
                  )
    new_txt = unidecode.unidecode(txt.lower().strip())
    res_arr = []
    for c in new_txt:
        if c in valid_chars:
            res_arr.append(c)
        else:
            res_arr.append(' ')
    res = ''.join(res_arr).strip()    
    return ' '.join(res.split())

def start(update: Update, context: CallbackContext) -> int:
    """"""
    
    return menu(update, context)


def menu(update: Update, context: CallbackContext) -> int:
    """Sends a message with three inline buttons attached."""
    keyboard = [
        [
            InlineKeyboardButton("Translate en->ru", callback_data='Translate en->ru'),
            InlineKeyboardButton("Translate ru->en", callback_data='Translate ru->en'),
        ],
        [InlineKeyboardButton("Synthesize en", callback_data='Synthesize en')],
        [InlineKeyboardButton("cancel", callback_data='cancel')],
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)

    update.message.reply_text('Please choose:', reply_markup=reply_markup)

    return ConversationHandler.END

def button(update: Update, context: CallbackContext) -> int:
    """Parses the CallbackQuery and updates the message text."""
    query = update.callback_query

    # CallbackQueries need to be answered, even if no notification to the user is needed
    # Some clients may have trouble otherwise. See https://core.telegram.org/bots/api#callbackquery
    query.answer()

    query.edit_message_text(text=f"Selected option: {query.data}")

    if query.data == "Translate en->ru":
        return TRANSLATE_EN_RU
    elif query.data == "Translate ru->en":
        return TRANSLATE_RU_EN
    elif query.data == "Synthesize en":
        return SYNTHESIZE_EN
    elif query.data == "cancel":
        return ConversationHandler.END


def help_command(update: Update, context: CallbackContext) -> None:
    """Displays info on how to use the bot."""
    update.message.reply_text("Use /start to test this bot.")

    
def get_message(update: Update) -> str:
    message = update.message.text
    
    message = message.replace('?', '&quest')
    
    return message
    
def translate_en_ru(update: Update, context: CallbackContext) -> None:
    user = update.message.from_user
    logger.info("Bio of %s: %s", user.first_name, update.message.text)
        
    message = get_message(update)
    request = requests.get(f'http://127.0.0.1:8000/translate/{message}?src_lang=en&trg_lang=ru')
    
    translation = request.json()['translation']
    
    update.message.reply_text(translation[0])                 
    
    
def translate_ru_en(update: Update, context: CallbackContext) -> None:
    user = update.message.from_user
    logger.info("Bio of %s: %s", user.first_name, update.message.text)
        
    message = get_message(update)
    request = requests.get(f'http://127.0.0.1:8000/translate/{message}?src_lang=ru&trg_lang=en')
    
    translation = request.json()['translation']
    
    update.message.reply_text(translation[0]) 
    
    
def synthesize_en(update: Update, context: CallbackContext) -> None:
    user = update.message.from_user
    logger.info("Bio of %s: %s", user.first_name, update.message.text)
        
    message = get_message(update)
    message = normalize_str(message)
    request = requests.get(f'http://127.0.0.1:8000/synthesize/{message}?src_lang=en')
    
    audio = request.json()['audio']
    
    audio_name = 'audio/' + message[:12] + '.wav'
    
    sf.write(audio_name, audio, 22050)
    
    context.bot.send_audio(chat_id=update.message.chat_id, audio=open(audio_name, 'rb'))
    
    
def cancel(update: Update, context: CallbackContext) -> int:
    """Cancels and ends the conversation."""
    user = update.message.from_user
    logger.info("User %s canceled the conversation.", user.first_name)
#     update.message.reply_text(
#         'Пока.', reply_markup=ReplyKeyboardRemove()
#     )

    return menu(update, context)

    
def main() -> None:
    """Run the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater(bot_token)

    updater.dispatcher.add_handler(CommandHandler('start', start))
#     updater.dispatcher.add_handler(CommandHandler('menu', menu))
#     updater.dispatcher.add_handler(CallbackQueryHandler(button))
    updater.dispatcher.add_handler(CommandHandler('help', help_command))
    updater.dispatcher.add_handler(ConversationHandler(
        entry_points=[CallbackQueryHandler(button)],
        states={
            TRANSLATE_EN_RU: [MessageHandler(Filters.text & ~Filters.command, translate_en_ru)],
            TRANSLATE_RU_EN: [MessageHandler(Filters.text & ~Filters.command, translate_ru_en)],
            SYNTHESIZE_EN: [MessageHandler(Filters.text & ~Filters.command, synthesize_en)],
        },
        fallbacks=[CommandHandler('menu', menu)],
    ))
    
    # Start the Bot
    updater.start_polling()

    # Run the bot until the user presses Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT
    updater.idle()


if __name__ == '__main__':
    main() 

/home/boris/anaconda3/lib/python3.7/site-packages/telegram/ext/conversationhandler.py:288: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  "If 'per_message=False', 'CallbackQueryHandler' will not be "
2022-01-28 03:43:15,559 - apscheduler.scheduler - INFO - Scheduler started
2022-01-28 03:43:24,411 - __main__ - INFO - Bio of Boris: Hello
2022-01-28 03:43:46,960 - __main__ - INFO - Bio of Boris: Audio example
2022-01-28 03:43:55,880 - __main__ - INFO - Bio of Boris: Can you help me with my job?
2022-01-28 03:44:30,149 - __main__ - INFO - Bio of Boris: I was swimming in the Caribbean.
Animals were hiding behind the rock.
Except the little fish bumped into me.
Swear, he was trying to talk to me, coy koi
